# Расчет установки по средней линии тока

### Импорт бибилиотек

In [16]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import json

### Функции для извлечения данных

In [115]:
def get_compressor_velocity_df(stage_dfs):
    dicts = []
    for stage_df in stage_dfs:
        dicts.append({
            'alpha_1': np.rad2deg(stage_df['inlet_triangle']['alpha']),
            'beta_1': np.rad2deg(stage_df['inlet_triangle']['beta']),
            'alpha_2': np.rad2deg(stage_df['mid_triangle']['alpha']),
            'beta_2': np.rad2deg(stage_df['mid_triangle']['beta']),
            'alpha_3': np.rad2deg(stage_df['outlet_triangle']['alpha']),
            'u': stage_df['inlet_triangle']['u'],
            'u_out': stage_df['u_out'],
            'c_a_1': stage_df['inlet_triangle']['ca'],
            'c_a_3': stage_df['outlet_triangle']['ca'],
        })
    return pd.DataFrame.from_records(dicts)

def format_compressor_velocity_df(df):
    local = df.copy()
    local.columns = [
        '$\\alpha_1, \\ град$', '$\\beta_1, \\ град$', '$\\alpha_2, \\ град$', 
        '$\\beta_2, \\ град$', '$\\alpha_3, \\ град$', '$c_{a1}, \\ м/c$', 
        '$c_{a3}, \\ м/c$', '$u, \\ м/c$', '$u_к, \\ м/c$'
    ]
    return local

def get_formatted_compressor_velocity_df(stage_dfs):
    return format_compressor_velocity_df(get_compressor_velocity_df(stage_dfs))

def get_compressor_thermo_df(stage_dfs):
    dicts = []
    for stage_df in stage_dfs:
        dicts.append({
            'h_t_coef': stage_df['heat_drop_coef'],
            'pi_stag': stage_df['pi_stag'],
            't1': stage_df['t_in'],
            'eta_ad': stage_df['eta_ad']
        })
    return pd.DataFrame.from_records(dicts)

def format_compressor_thermo_df(df):
    local = df.copy()
    local.columns= ['$\eta_{ад}$', '$\\overline{H_t}$', '$\\pi^*$', '$T_{0}, \\ К$']
    return local

def get_formatted_compressor_thermo_df(stage_dfs):
    return format_compressor_thermo_df(get_compressor_thermo_df(stage_dfs))

def get_geom_df(stage_dfs):
    dicts = []
    for stage_df in stage_dfs:
        geom = stage_df['stage_geometry']
        rotor = geom['rotor']
        stator = geom['stator']
        dicts.append({
            'h_rotor': rotor['blade_height'] * 1000,
            'w_rotor': rotor['blade_width'] * 1000,
            'h_stator': stator['blade_height'] * 1000,
            'w_stator': stator['blade_width'] * 1000,
            'gamma_in': np.rad2deg(rotor['gamma_in']),
            'gamma_out': np.rad2deg(rotor['gamma_out']),
        })
    return pd.DataFrame.from_records(dicts)

def format_geom_df(df):
    local = df.copy()
    local.columns = [
        '$\\gamma_{в}, \\ град$', '$\\gamma_{н}, \\ град$', '$h_с, \\ мм$', 
        '$h_р, \\ мм$', '$b_с, \\ мм$', '$b_р, \\ мм$'
    ]
    return local

def get_formatted_geom_df(stage_dfs):
    return format_geom_df(get_geom_df(stage_dfs))

def get_turbine_velocity_df(stage_dfs):
    dicts = []
    for stage_df in stage_dfs:
        dicts.append({
            'alpha_1': np.rad2deg(stage_df['rotor_inlet_triangle']['alpha']),
            'beta_1': np.rad2deg(stage_df['rotor_inlet_triangle']['beta']),
            'alpha_2': np.rad2deg(stage_df['rotor_outlet_triangle']['alpha']),
            'beta_2': np.rad2deg(stage_df['rotor_outlet_triangle']['beta']),
            'u': stage_df['rotor_inlet_triangle']['u'],
            'c_a_1': stage_df['rotor_inlet_triangle']['ca'],
            'c_a_2': stage_df['rotor_outlet_triangle']['ca'],
        })
    return pd.DataFrame.from_records(dicts)

def format_turbine_velocity_df(df):
    local = df.copy()
    local.columns = [
        '$\\alpha_1, \\ град$', '$\\beta_1, \\ град$', '$\\alpha_2, \\ град$', 
        '$\\beta_2, \\ град$', '$c_{a1}, \\ м/c$', '$c_{a2}, \\ м/c$', '$u, \\ м/c$'
    ]
    return local

def get_formatted_turbine_velocity_df(stage_dfs):
    return format_turbine_velocity_df(get_turbine_velocity_df(stage_dfs))

def get_turbine_thermo_df(stage_dfs):
    dicts = []
    for stage_df in stage_dfs:
        dicts.append({
            'phi': stage_df['phi'],
            'psi': stage_df['psi'],
            'reactivity': stage_df['reactivity'],
            'eta_t_stag': stage_df['eta_t_stag'],
            'pi': stage_df['pi'],
            't0': stage_df['t_0']
        })
    return pd.DataFrame.from_records(dicts)

def format_turbine_thermo_df(df):
    local = df.copy()
    local.columns= ['$\\eta^*$', '$\\phi$', '$\\pi^*$', '$\\psi$', '$r$', '$T_{0}, \\ К$']
    return local

def get_formatted_turbine_thermo_df(stage_dfs):
    return format_turbine_thermo_df(get_turbine_thermo_df(stage_dfs))

### Загрузка данных

In [82]:
data_root = "/home/artem/gowork/src/github.com/Sovianum/cooling-course-project/notebooks/data/staged.json"
data = json.load(open(data_root))

### Параметры КНД

#### Термодинамические парамеры ($\eta_{ад}$ - адиабатический КПД, $\overline{H_t}$ - коэффициент нагрузки, $\pi^*$ - степень повышения давления в компрессоре, $T_0$ - температура на входе, К)

In [103]:
get_formatted_compressor_thermo_df(data['lpc'])

,$\eta_{ад}$,$\overline{H_t}$,$\pi^*$,"$T_{0}, \ К$"
0,0.817514,0.290179,1.342627,288.000000
1,0.834198,0.308092,1.334701,318.719235
2,0.833531,0.318839,1.312029,351.264764
3,0.831529,0.322422,1.285112,384.838978
4,0.828192,0.322063,1.258785,418.637566
5,0.823520,0.320989,1.235886,452.131587
6,0.817514,0.319197,1.215661,485.269755


#### Скоростные параметры ($\alpha_1$, град - направление потока в абсолютном движении на входе в ротор, $\beta_1$, град - направление потока в относительном движении на входе в ротор, $\alpha_2$, град - направление потока в абсолютном движении на входе в статор, $\beta_2$, град - направление потока в относительном движении на входе в статор,  $\alpha_3$, град - направление потока в абсолютном движении на выходе из статора, $c_{a1}$, м/c - осевая скорость на входе в ротор, $c_{a3}$, м/c - осевая скорость на выходе из статора, $u$, м/c - окружная скорость на входе в ротор на среднем радиусе, $u_к$, м/c - окружная скорость на входе в ротор на периферии)

In [106]:
get_formatted_compressor_velocity_df(data['lpc'])

,"$\alpha_1, \ град$","$\beta_1, \ град$","$\alpha_2, \ град$","$\beta_2, \ град$","$\alpha_3, \ град$","$c_{a1}, \ м/c$","$c_{a3}, \ м/c$","$u, \ м/c$","$u_к, \ м/c$"
0,61.157584,39.471733,59.470842,39.572772,58.629354,164.461276,158.120322,289.563957,328.922552
1,59.470842,37.631250,58.196428,37.642771,57.644217,158.120322,154.402223,298.255076,328.922552
2,58.196428,36.202997,56.693248,36.461498,56.748042,154.402223,149.243717,304.702904,328.922552
3,56.693248,34.747998,54.835101,35.279193,55.380671,149.243717,142.021802,309.006828,328.922552
4,54.835101,33.560244,53.538449,33.844929,53.873195,142.021802,137.860806,311.845299,328.922552
5,53.538449,32.684544,52.308929,32.988624,52.709093,137.860806,133.846253,314.247859,328.922552
6,52.308929,31.582336,50.546560,32.184696,51.332141,133.846253,127.144498,316.085023,328.922552


#### Геометрические параметры ($\gamma_в$, град - угол раскрытия периферии, $\gamma_н$, град - угол раскрытия втулки, $h_с$, мм - высота лопатки статора, $h_р$, мм - высота лопатки ротора, $b_с$, мм - осевая проекция хорды статора, $b_р$, мм - осевая проекция хорды ротора)

In [108]:
get_formatted_geom_df(data['lpc'])

,"$\gamma_{в}, \ град$","$\gamma_{н}, \ град$","$h_с, \ мм$","$h_р, \ мм$","$b_с, \ мм$","$b_р, \ мм$"
0,22.0,0.0,201.920079,179.485302,50.480020,44.871325
1,20.0,0.0,159.543190,142.509625,42.544851,38.002567
2,16.0,0.0,127.294642,115.822851,36.369898,33.092243
3,12.0,0.0,105.384898,97.803275,32.426123,30.093315
4,11.0,0.0,90.767090,84.297867,30.255697,28.099289
5,9.0,0.0,78.289724,73.329774,28.468990,26.665372
6,6.0,0.0,68.684056,65.507703,27.473622,26.203081


### Параметры КВД

#### Термодинамические парамеры ($\eta_{ад}$ - адиабатический КПД, $\overline{H_t}$ - коэффициент нагрузки, $\pi^*$ - степень повышения давления в компрессоре, $T_0$ - температура на входе, К)

In [109]:
get_formatted_compressor_thermo_df(data['hpc'])

,$\eta_{ад}$,$\overline{H_t}$,$\pi^*$,"$T_{0}, \ К$"
0,0.798646,0.267635,1.336662,521.656793
1,0.810870,0.271732,1.312539,575.454107
2,0.814945,0.273097,1.286480,629.484316
3,0.810870,0.271732,1.259294,683.164503
4,0.798646,0.267635,1.231596,735.917636


#### Скоростные параметры ($\alpha_1$, град - направление потока в абсолютном движении на входе в ротор, $\beta_1$, град - направление потока в относительном движении на входе в ротор, $\alpha_2$, град - направление потока в абсолютном движении на входе в статор, $\beta_2$, град - направление потока в относительном движении на входе в статор,  $\alpha_3$, град - направление потока в абсолютном движении на выходе из статора, $c_{a1}$, м/c - осевая скорость на входе в ротор, $c_{a3}$, м/c - осевая скорость на выходе из статора, $u$, м/c - окружная скорость на входе в ротор на среднем радиусе, $u_к$, м/c - окружная скорость на входе в ротор на периферии)

In [110]:
get_formatted_compressor_velocity_df(data['hpc'])

,"$\alpha_1, \ град$","$\beta_1, \ град$","$\alpha_2, \ град$","$\beta_2, \ град$","$\alpha_3, \ град$","$c_{a1}, \ м/c$","$c_{a3}, \ м/c$","$u, \ м/c$","$u_к, \ м/c$"
0,90.000000,55.142165,49.159364,22.582436,30.110209,184.921534,183.440880,444.629590,462.303834
1,49.159364,31.636206,47.002702,32.451891,47.874248,183.440880,171.151846,447.047839,462.303834
2,47.002702,30.064170,45.404532,30.587711,46.070943,171.151846,163.356647,449.130269,462.303834
3,45.404532,29.094612,44.226985,29.404446,44.760904,163.356647,158.673678,450.924804,462.303834
4,44.226985,28.414312,42.980136,28.732194,43.639624,158.673678,154.096407,452.451233,462.303834


#### Геометрические параметры ($\gamma_в$, град - угол раскрытия периферии, $\gamma_н$, град - угол раскрытия втулки, $h_с$, мм - высота лопатки статора, $h_р$, мм - высота лопатки ротора, $b_с$, мм - осевая проекция хорды статора, $b_р$, мм - осевая проекция хорды ротора)

In [111]:
get_formatted_geom_df(data['hpc'])

,"$\gamma_{в}, \ град$","$\gamma_{н}, \ град$","$h_с, \ мм$","$h_р, \ мм$","$b_с, \ мм$","$b_р, \ мм$"
0,9.0,0.0,55.183455,51.337767,22.073382,20.535107
1,9.0,0.0,47.760081,44.431722,19.104033,17.772689
2,9.0,0.0,41.335313,38.454690,16.534125,15.381876
3,8.0,0.0,35.774815,33.316768,15.899918,14.807453
4,7.0,0.0,31.027612,29.165086,13.790050,12.962260


### Параметры ТВД

#### Термодинамические параметры ($\eta^*$ - КПД по параметрам торможения, $\phi$ - коэффициент скорости статора, $\psi$ - коэффициент скорости ротора, $\pi^*$ - степень понижения давления, $r$ - степень реактивности ступени, $T_0$, К  - температура газа на входе)

In [116]:
get_formatted_turbine_thermo_df(data['hpt'])

,$\eta^*$,$\phi$,$\pi^*$,$\psi$,$r$,"$T_{0}, \ К$"
0,0.880113,0.939702,2.804824,0.939702,0.3,1450


#### Скоростные параметры ($\alpha_1$, град - направление потока в абсолютном движении на входе в ротор, $\beta_1$, град - направление потока в относительном движении на входе в ротор, $\alpha_2$, град - направление потока в абсолютном движении на выходе из ротора, $\beta_2$, град - направление потока в относительном движении на выходе из ротора, $c_{a1}$, м/c - осевая скорость на входе в ротор, $c_{a2}$, м/c - осевая скорость на выходе из ротора, $u$, м/c - окружная скорость на входе в ротор на среднем радиусе)

In [118]:
get_formatted_turbine_velocity_df(data['hpt'])

,"$\alpha_1, \ град$","$\beta_1, \ град$","$\alpha_2, \ град$","$\beta_2, \ град$","$c_{a1}, \ м/c$","$c_{a2}, \ м/c$","$u, \ м/c$"
0,12.0,88.354542,35.557623,17.482608,144.97466,155.000499,479.236939


#### Геометрические параметры ($\gamma_в$, град - угол раскрытия периферии, $\gamma_н$, град - угол раскрытия втулки, $h_с$, мм - высота лопатки статора, $h_р$, мм - высота лопатки ротора, $b_с$, мм - осевая проекция хорды статора, $b_р$, мм - осевая проекция хорды ротора)

In [120]:
get_formatted_geom_df(data['hpt'])

,"$\gamma_{в}, \ град$","$\gamma_{н}, \ град$","$h_с, \ мм$","$h_р, \ мм$","$b_с, \ мм$","$b_р, \ мм$"
0,0.0,12.0,87.003065,67.959914,57.382777,63.741036


### Параметры ТНД

#### Термодинамические параметры ($\eta^*$ - КПД по параметрам торможения, $\phi$ - коэффициент скорости статора, $\psi$ - коэффициент скорости ротора, $\pi^*$ - степень понижения давления, $r$ - степень реактивности ступени, $T_0$, К  - температура газа на входе)

In [121]:
get_formatted_turbine_thermo_df(data['lpt'])

,$\eta^*$,$\phi$,$\pi^*$,$\psi$,$r$,"$T_{0}, \ К$"
0,0.896548,0.95015,1.554595,0.95015,0.5,1178.939045
1,0.894089,0.95015,1.625226,0.95015,0.5,1069.678814


#### Скоростные параметры ($\alpha_1$, град - направление потока в абсолютном движении на входе в ротор, $\beta_1$, град - направление потока в относительном движении на входе в ротор, $\alpha_2$, град - направление потока в абсолютном движении на выходе из ротора, $\beta_2$, град - направление потока в относительном движении на выходе из ротора, $c_{a1}$, м/c - осевая скорость на входе в ротор, $c_{a2}$, м/c - осевая скорость на выходе из ротора, $u$, м/c - окружная скорость на входе в ротор на среднем радиусе)

In [122]:
get_formatted_turbine_velocity_df(data['lpt'])

,"$\alpha_1, \ град$","$\beta_1, \ град$","$\alpha_2, \ град$","$\beta_2, \ град$","$c_{a1}, \ м/c$","$c_{a2}, \ м/c$","$u, \ м/c$"
0,16.00000,69.453120,71.971139,17.123813,99.511532,110.853171,314.649271
1,19.23195,77.499941,81.683593,20.264335,118.918652,132.194584,323.492563


#### Геометрические параметры ($\gamma_в$, град - угол раскрытия периферии, $\gamma_н$, град - угол раскрытия втулки, $h_с$, мм - высота лопатки статора, $h_р$, мм - высота лопатки ротора, $b_с$, мм - осевая проекция хорды статора, $b_р$, мм - осевая проекция хорды ротора)

In [123]:
get_formatted_geom_df(data['lpt'])

,"$\gamma_{в}, \ град$","$\gamma_{н}, \ град$","$h_с, \ мм$","$h_р, \ мм$","$b_с, \ мм$","$b_р, \ мм$"
0,0.0,8.0,161.843982,151.236672,63.582800,64.482112
1,0.0,10.0,185.200710,169.995236,73.901728,73.717374


### Параметры ТС

#### Термодинамические параметры ($\eta^*$ - КПД по параметрам торможения, $\phi$ - коэффициент скорости статора, $\psi$ - коэффициент скорости ротора, $\pi^*$ - степень понижения давления, $r$ - степень реактивности ступени, $T_0$, К  - температура газа на входе)

In [124]:
get_formatted_turbine_thermo_df(data['ft'])

,$\eta^*$,$\phi$,$\pi^*$,$\psi$,$r$,"$T_{0}, \ К$"
0,0.917674,0.963577,1.410970,0.963577,0.4,972.895380
1,0.913986,0.963577,1.449760,0.963577,0.4,897.368350
2,0.907880,0.963577,1.495293,0.963577,0.4,828.084591


#### Скоростные параметры ($\alpha_1$, град - направление потока в абсолютном движении на входе в ротор, $\beta_1$, град - направление потока в относительном движении на входе в ротор, $\alpha_2$, град - направление потока в абсолютном движении на выходе из ротора, $\beta_2$, град - направление потока в относительном движении на выходе из ротора, $c_{a1}$, м/c - осевая скорость на входе в ротор, $c_{a2}$, м/c - осевая скорость на выходе из ротора, $u$, м/c - окружная скорость на входе в ротор на среднем радиусе)

In [125]:
get_formatted_turbine_velocity_df(data['ft'])

,"$\alpha_1, \ град$","$\beta_1, \ град$","$\alpha_2, \ град$","$\beta_2, \ град$","$c_{a1}, \ м/c$","$c_{a2}, \ м/c$","$u, \ м/c$"
0,22.000000,89.546584,71.765214,25.624469,121.219929,126.852935,260.093200
1,24.885358,94.014197,79.182066,28.794195,136.169070,143.564471,267.527743
2,28.525336,97.820539,86.845310,32.555265,154.530790,164.327900,275.793396


#### Геометрические параметры ($\gamma_в$, град - угол раскрытия периферии, $\gamma_н$, град - угол раскрытия втулки, $h_с$, мм - высота лопатки статора, $h_р$, мм - высота лопатки ротора, $b_с$, мм - осевая проекция хорды статора, $b_р$, мм - осевая проекция хорды ротора)

In [126]:
get_formatted_geom_df(data['ft'])

,"$\gamma_{в}, \ град$","$\gamma_{н}, \ град$","$h_с, \ мм$","$h_р, \ мм$","$b_с, \ мм$","$b_р, \ мм$"
0,0.0,10.0,238.685542,224.232653,62.712300,67.825142
1,0.0,10.0,265.342888,249.313256,69.716258,75.411438
2,0.0,10.0,294.982682,277.206108,77.503825,83.848375
